In [1]:
!pip install sqlalchemy
!pip install PyPDF2
!pip install textract
!pip install tika 
!pip install bs4

Looking in indexes: https://****@pkgs.dev.azure.com/msci-otw/_packaging/msci-otw-org-feed/pypi/simple
Looking in indexes: https://****@pkgs.dev.azure.com/msci-otw/_packaging/msci-otw-org-feed/pypi/simple
Looking in indexes: https://****@pkgs.dev.azure.com/msci-otw/_packaging/msci-otw-org-feed/pypi/simple
Looking in indexes: https://****@pkgs.dev.azure.com/msci-otw/_packaging/msci-otw-org-feed/pypi/simple
Looking in indexes: https://****@pkgs.dev.azure.com/msci-otw/_packaging/msci-otw-org-feed/pypi/simple


In [2]:
import numpy as np
import requests
import time
import msci
import tika
from tika import parser
from datetime import date, datetime, timedelta
import PyPDF2 
import textract
import base64
import binascii
import io
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import cx_Oracle
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from msci.data import get, get_source  # main data fetching function, consult the data dictionary from the Help
from msci.esg_api.client.ESGDataAPIClient import ESGDataAPIClient
from msci.esg_api import __version__
import logging
logging.getLogger('msci.data.user_warnings').setLevel(logging.ERROR)

In [3]:
# Database connection parameters
username = "CAPTURE"
password = "Capture_#prod_1"
host = "pva2es1lora0001.int.msci.com"
port = "1524"
service_name = "P_CAP01"

# Create a SQLAlchemy engine
engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}')

# Create a session
Session = sessionmaker(bind=engine)
session = Session()



In [4]:

def get_doc(doc_id):
    with requests.Session() as session:
        
        data = None
        try:
            response = session.get(DOCUMENTS_URL + f'{doc_id}?includeDocument=true')
            response.raise_for_status()
            if response.headers.get('Content-Type') == 'application/json':
                resp_json = response.json()
                data = resp_json.get('data')
            else:
                print(f"Unexpected content type: {response.headers.get('Content-Type')}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
        except ValueError as e:
            print(f"JSON decode failed: {e}")
    if data is None:
        print(f'Failed Retrieval - Filing ID: {doc_id}')
        return None
    return binascii.a2b_base64(data)

def searchInPDF(x, doc_type, keyword):
    result = []
    try:
        if doc_type == '1':
            text = parser.from_file(x, xmlContent=True)
        else:
            text = parser.from_buffer(x, xmlContent=True)
        xhtml_data = BeautifulSoup(text["content"], 'html.parser')
        for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
            hits = re.search(keyword, str(content), re.IGNORECASE)
            if hits:
                result.append(page + 1)
    except Exception as e:
        print(f"Error processing PDF content: {e}")
    return {'keyword': keyword, 'page_hits': result}

def searchkeyHTML(html_content, keyword):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text()
        keyword_present = bool(re.search(keyword, text_content, re.IGNORECASE))
        return {'keyword': keyword, 'keyword_present': keyword_present}
    except Exception as e:
        print(f'Error processing HTML content: {e}')
        return {'keyword': keyword, 'keyword_present': False}

def detect_doc_type(doc_id):
    try:
        # Fetch metadata
        response = requests.get(DOCUMENTS_URL + f'{doc_id}?includeDocument=true')
        response.raise_for_status()
        metadata = response.json()
        
        # Extract document format from metadata
        document_format = metadata.get('documentFormat', '').lower()
        
        # Determine document type
        if document_format == 'pdf':
            return 'pdf'
        elif document_format == 'html':
            return 'html'
        elif document_format == 'xml':
            return 'xml'
        else:
            return 'unknown'
    
    except requests.RequestException as e:
        print(f"Error fetching metadata for document ID {doc_id}: {e}")
        return 'unknown'
    except ValueError as e:
        print(f"Error decoding JSON for document ID {doc_id}: {e}")
        return 'unknown'

In [ ]:
# Start the timer
start_time = time.time()

# Query to fetch data
SQL = '''SELECT DMX_ISSUER_ID, DMX_ISSUER_NAME, COV_EXPOSURE FROM COVERAGE_INDEX_VW WHERE (("COV_EXPOSURE" = '1') AND (is_restricted = 0))'''
COV_EXP = pd.read_sql(SQL, engine)
EXP = COV_EXP[0:1000]
print('Connection Successful')


# Define the fiscal_year and keywords
fiscal_year = 2023
keywords = ['fuel sales', 'oil', 'gas', 'petroleum', 'liquid transportation']
DOCUMENTS_URL = 'http://dap-docstore.msciapps.com/documentStore/v1/document/'


issuer_ids = EXP['dmx_issuer_id'].tolist()  # Extract issuer_ids from the dataframe
issuer_ids = [id.strip() for id in issuer_ids]  # Clean any extra whitespace
all_results = []  # To store results for all issuer IDs
# Initialize a counter
counter = 1
for issuer_id in issuer_ids:
    print(f"Processing document set {counter} for Issuer ID {issuer_id}...")
    get_doc_id_list = f"""
    SELECT d.id, d.fiscal_year, d.original_source, dt.type_name
    FROM documents.document d LEFT JOIN documents.document_type dt
    ON d.document_type_id  = dt.ID
    WHERE d.document_language_id = 1
      AND d.approval_status_type_id IN (2)
      AND fiscal_year in {fiscal_year}
      AND d.document_type_id IN (1,9,10,11,14,15,16,17,43,45,107,130,131,132)
      AND issuer_id = :issuer_id
    """
    get_doc_id_list_raw = pd.read_sql(get_doc_id_list, engine, params={'issuer_id': issuer_id})
    doc_id_list = get_doc_id_list_raw['id'].tolist()

    if not doc_id_list:
        print(f"No documents found for Issuer ID {issuer_id} and fiscal year {fiscal_year}")
    else:
        for j, doc_id in enumerate(doc_id_list):
            doc_info = get_doc_id_list_raw[get_doc_id_list_raw['id'] == doc_id].iloc[0]
            fiscal_year = doc_info['fiscal_year']
            type_name = doc_info['type_name']
            original_source = doc_info['original_source']
            print(f"Processing document {j + 1} of {len(doc_id_list)} for Issuer ID {issuer_id}")
            content = get_doc(doc_id)
            if content:
                doc_type_detected = detect_doc_type(doc_id)
                print(f"Document ID {doc_id} detected as {doc_type_detected}")
                if doc_type_detected == 'pdf':
                    for keyword in keywords:
                        result = searchInPDF(content, doc_type_detected, keyword)
                        temp_out = pd.DataFrame(index=[0])
                        temp_out['source'] = doc_id
                        temp_out['Issuer_ID'] = issuer_id
                        temp_out['keyword'] = keyword
                        temp_out['page_hits'] = str(result['page_hits'])
                        temp_out['keyword_present'] = "N/A"
                        temp_out['fiscal_year'] = fiscal_year
                        temp_out['doc_type'] = type_name
                        temp_out['link'] = original_source
                        all_results.append(temp_out)
                elif doc_type_detected == 'html':
                    html_content = content.decode('utf-8', errors='ignore')
                    #print(f"HTML DOC: {doc_id}")
                    for keyword in keywords:
                        result = searchkeyHTML(html_content, keyword)
                        temp_out = pd.DataFrame(index=[0])
                        temp_out['source'] = doc_id
                        temp_out['Issuer_ID'] = issuer_id
                        temp_out['keyword'] = keyword
                        temp_out['page_hits'] = "HTML document"
                        temp_out['keyword_present'] = result['keyword_present']
                        temp_out['fiscal_year'] = fiscal_year
                        temp_out['doc_type'] = type_name
                        temp_out['link'] = original_source
                        all_results.append(temp_out)
                else:
                    print(f"Unknown document type for Document ID {doc_id}")
            else:
                print(f"Failed to retrieve content for Document ID {doc_id}")
                        
# Print the counter value for debugging
    print(f"Completed processing for Issuer ID {issuer_id} (Set {counter})")
    
    # Increment the counter after processing each issuer ID
    counter += 1
                        
                        
final_out = pd.concat(all_results, ignore_index=True)
final_out.to_csv("/home/jovyan/work/shared/Rushikesh Tambe/Keyword_output_2" + pd.Timestamp.now().strftime("%Y%m%d") + ".csv", index=False)
print("Keyword search complete")


# End the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
elapsed_minutes = elapsed_time / 60  # Convert to minutes
print(f"Script ran for {elapsed_minutes:.2f} minutes")

Connection Successful
Processing document set 1 for Issuer ID IID000000002123682...
Processing document 1 of 1 for Issuer ID IID000000002123682
Document ID 3841898 detected as html
Completed processing for Issuer ID IID000000002123682 (Set 1)
Processing document set 2 for Issuer ID IID000000002123684...
Processing document 1 of 1 for Issuer ID IID000000002123684
Document ID 3656538 detected as html
Completed processing for Issuer ID IID000000002123684 (Set 2)
Processing document set 3 for Issuer ID IID000000002123685...
Processing document 1 of 1 for Issuer ID IID000000002123685
Document ID 3947293 detected as html
Completed processing for Issuer ID IID000000002123685 (Set 3)
Processing document set 4 for Issuer ID IID000000002123686...
Processing document 1 of 1 for Issuer ID IID000000002123686
Document ID 3860117 detected as html
Completed processing for Issuer ID IID000000002123686 (Set 4)
Processing document set 5 for Issuer ID IID000000002123687...
Processing document 1 of 1 for I

2024-07-31 03:07:37,217 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Document ID 4090276 detected as pdf
Completed processing for Issuer ID IID000000002123750 (Set 38)
Processing document set 39 for Issuer ID IID000000002123757...
Processing document 1 of 1 for Issuer ID IID000000002123757
Document ID 4190589 detected as pdf
Completed processing for Issuer ID IID000000002123757 (Set 39)
Processing document set 40 for Issuer ID IID000000002123759...
Processing document 1 of 2 for Issuer ID IID000000002123759
Document ID 3718394 detected as pdf
Processing document 2 of 2 for Issuer ID IID000000002123759
Document ID 3683859 detected as pdf
Completed processing for Issuer ID IID000000002123759 (Set 40)
Processing document set 41 for Issuer ID IID000000002123767...
Processing document 1 of 1 for Issuer ID IID000000002123767
Document ID 3620726 detected as pdf
Completed processing for Issuer ID IID000000002123767 (Set 41)
Processing document set 42 for Issuer ID IID000000002123768...
Processing document 1 of 1 for Issuer ID IID000000002123768
Document ID 4126

2024-07-31 04:47:55,075 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Document ID 3525379 detected as pdf
Processing document 2 of 2 for Issuer ID IID000000002124584
Document ID 3414465 detected as pdf
Completed processing for Issuer ID IID000000002124584 (Set 176)
Processing document set 177 for Issuer ID IID000000002124585...
Processing document 1 of 3 for Issuer ID IID000000002124585
Document ID 3292542 detected as pdf
Processing document 2 of 3 for Issuer ID IID000000002124585
Document ID 3761374 detected as pdf
Processing document 3 of 3 for Issuer ID IID000000002124585
Document ID 3756607 detected as pdf
Completed processing for Issuer ID IID000000002124585 (Set 177)
Processing document set 178 for Issuer ID IID000000002124592...
No documents found for Issuer ID IID000000002124592 and fiscal year 2023
Completed processing for Issuer ID IID000000002124592 (Set 178)
Processing document set 179 for Issuer ID IID000000002124593...
Processing document 1 of 2 for Issuer ID IID000000002124593
Document ID 4162504 detected as pdf
Processing document 2 of 2 

...

Document ID 3863410 detected as pdf
Completed processing for Issuer ID IID000000002124731 (Set 208)
Processing document set 209 for Issuer ID IID000000002124736...
Processing document 1 of 4 for Issuer ID IID000000002124736
Document ID 4137058 detected as pdf
Processing document 2 of 4 for Issuer ID IID000000002124736
Document ID 3863439 detected as pdf
Processing document 3 of 4 for Issuer ID IID000000002124736
Document ID 3970321 detected as html
Processing document 4 of 4 for Issuer ID IID000000002124736
Document ID 3974566 detected as pdf
Completed processing for Issuer ID IID000000002124736 (Set 209)
Processing document set 210 for Issuer ID IID000000002124737...
Processing document 1 of 1 for Issuer ID IID000000002124737
Document ID 3845509 detected as html
Completed processing for Issuer ID IID000000002124737 (Set 210)
Processing document set 211 for Issuer ID IID000000002124746...
Processing document 1 of 2 for Issuer ID IID000000002124746
Document ID 3159109 detected as html
P

2024-07-31 12:57:21,714 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Document ID 4000569 detected as pdf
Completed processing for Issuer ID IID000000002131803 (Set 779)
Processing document set 780 for Issuer ID IID000000002131818...
No documents found for Issuer ID IID000000002131818 and fiscal year 2023
Completed processing for Issuer ID IID000000002131818 (Set 780)
Processing document set 781 for Issuer ID IID000000002131851...
Processing document 1 of 1 for Issuer ID IID000000002131851
Document ID 3987770 detected as pdf
Completed processing for Issuer ID IID000000002131851 (Set 781)
Processing document set 782 for Issuer ID IID000000002131855...
Processing document 1 of 1 for Issuer ID IID000000002131855
Document ID 3239205 detected as pdf
Completed processing for Issuer ID IID000000002131855 (Set 782)
Processing document set 783 for Issuer ID IID000000002131860...
Processing document 1 of 2 for Issuer ID IID000000002131860
Document ID 4114036 detected as pdf
Processing document 2 of 2 for Issuer ID IID000000002131860
Document ID 3837533 detected as

2024-07-31 13:54:36,886 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Document ID 3468399 detected as pdf
Processing document 2 of 2 for Issuer ID IID000000002132267
Document ID 3487828 detected as pdf
Completed processing for Issuer ID IID000000002132267 (Set 876)
Processing document set 877 for Issuer ID IID000000002132268...
Processing document 1 of 1 for Issuer ID IID000000002132268
Document ID 3845747 detected as pdf
Completed processing for Issuer ID IID000000002132268 (Set 877)
Processing document set 878 for Issuer ID IID000000002132272...
Processing document 1 of 2 for Issuer ID IID000000002132272
Document ID 3846946 detected as pdf
Processing document 2 of 2 for Issuer ID IID000000002132272
Document ID 3838068 detected as pdf
Completed processing for Issuer ID IID000000002132272 (Set 878)
Processing document set 879 for Issuer ID IID000000002132273...
Processing document 1 of 1 for Issuer ID IID000000002132273
Document ID 3963476 detected as pdf
Completed processing for Issuer ID IID000000002132273 (Set 879)
Processing document set 880 for Issu